In [1]:
import pandas as pd
import numpy as np

In [2]:
data_abund = pd.read_csv("9606_abund.txt", sep="\t")

In [3]:
data_dom = pd.read_csv("9606_gn_dom.txt", sep="\t")

In [4]:
data_abund = data_abund.rename(columns={"#Taxid": "Taxid"})
data_dom = data_dom.rename(columns={"#Gn": "Gn"})

In [5]:
data_abund.head(10)

,Taxid,Ensembl_protein,Gn,Mean-copy-number
0,9606,ENSP00000263100,A1BG,885.188
1,9606,ENSP00000282641,A1CF,19.016
2,9606,ENSP00000282641,A1CF,19.016
3,9606,ENSP00000282641,A1CF,19.016
4,9606,ENSP00000323929,A2M,1114.564
5,9606,ENSP00000323929,A2M,1114.564
6,9606,ENSP00000323929,A2M,1114.564
7,9606,ENSP00000323929,A2M,1114.564
8,9606,ENSP00000299698,A2ML1,90.762
9,9606,ENSP00000299698,A2ML1,90.762


In [6]:
data_dom.head(10)

,Gn,Domain,Start,End,Eval
0,A1BG,Ig,127,201,3.800000e-01
1,A1BG,Ig,217,300,3.000000e-15
2,A1BG,Ig,31,110,8.200000e-06
3,A1BG,Ig,403,490,1.900000e-03
4,A1BG,SpaA,327,352,4.400000e+01
5,A1CF,DND1_DSRM,447,523,2.300000e-24
6,A1CF,RRM,138,199,4.400000e-07
7,A1CF,RRM,233,296,6.700000e-11
8,A1CF,RRM,58,124,2.400000e-16
9,A2M,A2M,738,828,4.500000e-31


## A1

### How many protein/copy-number pairs are in the file? (Single numerical value)

In [7]:
data_abund.shape[0]

53642

### How many unique copy number values are there in the file?

In [8]:
data_abund["Mean-copy-number"].nunique()

16241

### How many pairs of protein and copy number values are in the file? (Single numerical value)

In [9]:
data_abund.drop_duplicates().shape[0]

19572

## A2. Compute the mean and standard deviation of copy numbers for all proteins (considering unique pairs only) first as a single number for all proteins (two numerical values) and then for each protein separately (Table in tsv/csv).

In [10]:
#drop duplicates
unique_copy_num = data_abund.drop_duplicates()
unique_copy_num.shape

(19572, 4)

In [11]:
unique_copy_num.isna().sum()

Taxid               0
Ensembl_protein     0
Gn                  0
Mean-copy-number    0
dtype: int64

In [12]:
#convert format to numerical
unique_copy_num["Mean-copy-number"] = pd.to_numeric(unique_copy_num["Mean-copy-number"], errors="coerce")
mean_value = unique_copy_num["Mean-copy-number"].mean()

/tmp/ipykernel_370219/487506573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_copy_num["Mean-copy-number"] = pd.to_numeric(unique_copy_num["Mean-copy-number"], errors="coerce")


In [13]:
std_value = unique_copy_num["Mean-copy-number"].std()

In [14]:
print(f"Mean copy number: {mean_value:.2f}")
print(f"Standard Deviation copy number: {std_value:.2f}")

Mean copy number: 79.83
Standard Deviation copy number: 362.17


In [15]:
mean_std_for_each_protein = unique_copy_num.groupby("Gn")["Mean-copy-number"].agg(["mean", "std"]).reset_index()

In [16]:
mean_std_for_each_protein.count()

Gn      18992
mean    18991
std       468
dtype: int64

In [17]:
mean_std_for_each_protein.isna().sum()

Gn          0
mean        1
std     18524
dtype: int64

In [18]:
mean_std_for_each_protein.sort_values("mean", ascending=False).head(10)

,Gn,mean,std
531,ALB,22306.386,NaN
6853,HBA2,14178.655,NaN
6854,HBB,13538.518,NaN
8428,LALBA,12454.998,NaN
16847,TMSB4X,11622.015,NaN
7435,IGLJ1,7235.736,NaN
7437,IGLL5,7235.736,NaN
7434,IGLC1,7235.736,NaN
3682,CSN1S1,7160.290,NaN
6037,GAPDH,6999.011,NaN


In [19]:
mean_std_for_each_protein.sort_values("mean", ascending=False).to_csv("A2task_mean_std_proteins_abund.csv", index=False)

### A3. Calculate the percentile rank (in terms of average copy number ranks) for each protein. (i.e. for protein X, where is it in the ranks from top (0%) to bottom (100%) in terms of abundance) (Table in csv/tsv). Please also give the top ten proteins (highest abundance) as a list with the associated numerical values.

In [20]:
mean_std_for_each_protein["percentile_rank"] = mean_std_for_each_protein["mean"].rank(pct=True, ascending=True) * 100

In [24]:
mean_std_for_each_protein.sort_values("percentile_rank", ascending=False).to_csv("A3task_mean_std_rank.csv")

In [21]:
top_10 = mean_std_for_each_protein.sort_values("percentile_rank", ascending=False).head(10)

In [22]:
top_10

,Gn,mean,std,percentile_rank
531,ALB,22306.386,NaN,100.000000
6853,HBA2,14178.655,NaN,99.994734
6854,HBB,13538.518,NaN,99.989469
8428,LALBA,12454.998,NaN,99.984203
16847,TMSB4X,11622.015,NaN,99.978937
7435,IGLJ1,7235.736,NaN,99.968406
7437,IGLL5,7235.736,NaN,99.968406
7434,IGLC1,7235.736,NaN,99.968406
3682,CSN1S1,7160.290,NaN,99.957875
6037,GAPDH,6999.011,NaN,99.952609


In [23]:
top_10[["Gn", "mean"]].values.tolist()

[['ALB', 22306.386],
 ['HBA2', 14178.655],
 ['HBB', 13538.518],
 ['LALBA', 12454.998],
 ['TMSB4X', 11622.015],
 ['IGLJ1', 7235.736],
 ['IGLL5', 7235.736],
 ['IGLC1', 7235.736],
 ['CSN1S1', 7160.29],
 ['GAPDH', 6999.011]]

# Analyse protein domains

### B1. What is the domain with the highest average abundance (i.e. across all copies of the domain in all proteins) and what is the value of the average abundance, and how many times was the domain seen? (single string value and two numerical values)

In [23]:
data_dom.head(5)

,Gn,Domain,Start,End,Eval
0,A1BG,Ig,127,201,3.800000e-01
1,A1BG,Ig,217,300,3.000000e-15
2,A1BG,Ig,31,110,8.200000e-06
3,A1BG,Ig,403,490,1.900000e-03
4,A1BG,SpaA,327,352,4.400000e+01


In [24]:
data_dom.isna().sum()

Gn        0
Domain    0
Start     0
End       0
Eval      0
dtype: int64

In [25]:
data_dom.shape

(65884, 5)

In [26]:
data_dom.drop_duplicates().shape

(65877, 5)

In [27]:
data_dom = data_dom.drop_duplicates()

In [28]:
proteins_domains = data_dom.merge(mean_std_for_each_protein[["Gn", "mean", "std"]], on="Gn", how="left")

In [29]:
# domain average abundance (i.e. across all copies of the domain in all proteins)
count_domain = proteins_domains.groupby(["Gn","Domain","mean","std"])["Domain"].size().reset_index(name="count_domain")

In [30]:
count_domain

,Gn,Domain,mean,std,count_domain
0,AAK1,Pkinase,42.888500,0.101116,1
1,AARSD1,tRNA-synt_2c,33.221000,2.791658,1
2,AARSD1,tRNA_SAD,33.221000,2.791658,1
3,ACE,Peptidase_M2,18.898333,3.137940,2
4,ACTR3C,Actin,30.587500,3.796456,1
...,...,...,...,...,...
743,ZNF83,zf-C2H2,1.976000,1.415452,26
744,ZNF83,zf-H2C2,1.976000,1.415452,14
745,ZNF84,KRAB,1.407000,0.000000,1
746,ZNF84,zf-C2H2,1.407000,0.000000,37


In [31]:
# domain with the highest average abundance (i.e. across all copies of the domain in all proteins)
count_domain["domain_average_abundance"] = count_domain["count_domain"] * count_domain["mean"]

In [32]:
count_domain

,Gn,Domain,mean,std,count_domain,domain_average_abundance
0,AAK1,Pkinase,42.888500,0.101116,1,42.888500
1,AARSD1,tRNA-synt_2c,33.221000,2.791658,1,33.221000
2,AARSD1,tRNA_SAD,33.221000,2.791658,1,33.221000
3,ACE,Peptidase_M2,18.898333,3.137940,2,37.796667
4,ACTR3C,Actin,30.587500,3.796456,1,30.587500
...,...,...,...,...,...,...
743,ZNF83,zf-C2H2,1.976000,1.415452,26,51.376000
744,ZNF83,zf-H2C2,1.976000,1.415452,14,27.664000
745,ZNF84,KRAB,1.407000,0.000000,1,1.407000
746,ZNF84,zf-C2H2,1.407000,0.000000,37,52.059000


In [33]:
most_prevalent_domain = count_domain.groupby("Domain").agg({
    "domain_average_abundance": "sum",
    "count_domain": "sum"
}).reset_index().sort_values("domain_average_abundance", ascending=False).head(1)

In [35]:
domain_name = most_prevalent_domain["Domain"].iloc[0]
avg_abundance = most_prevalent_domain["domain_average_abundance"].iloc[0]
times_seen = most_prevalent_domain["count_domain"].iloc[0]

print(f"domain with the highest average abundance"
      f"(i.e. across all copies of the domain in all proteins): {domain_name}, {round(avg_abundance, 2)}, {times_seen}")

domain with the highest average abundance(i.e. across all copies of the domain in all proteins): ubiquitin, 9970.83, 14


In [36]:
# top 10 abundant domains
count_domain.groupby("Domain").agg({
    "domain_average_abundance": "sum",
    "count_domain": "sum"
}).reset_index().sort_values("domain_average_abundance", ascending=False).head(10)

,Domain,domain_average_abundance,count_domain
422,ubiquitin,9970.828250,14
386,Tropomyosin,4653.332000,3
388,Tubulin,4617.474333,7
389,Tubulin_C,4617.474333,7
425,zf-C2H2,2261.611833,620
362,Spectrin,2088.733500,31
319,RRM,1948.366333,15
115,EF-hand,1634.424000,6
390,UDP-g_GGTase,1621.509000,2
369,Sushi,1439.700500,5


### B2. Compute the mean and standard deviation of domain average abundance for each protein domain (i.e. by summing abundance values of all versions of these domains) by combining these two files also, compute the percentile rank values as above (One table)

### To calculate Std we need to use propagation of error formula:

$
\sigma = \sqrt{\sigma_1^2 + \sigma_2^2 + \dots + \sigma_n^2}
$

In [37]:
std_count = (
    count_domain.assign(std_squared=lambda d: d["std"]**2)
    .groupby("Domain")["std_squared"]
    .sum()
    .apply(np.sqrt)
    .reset_index()
    .rename(columns={"std_squared": "domain_abundance_std"})
)

In [38]:
domain_summary = count_domain.groupby("Domain").agg({
    "domain_average_abundance": "sum",
    "count_domain": "sum"
}).reset_index().sort_values("domain_average_abundance", ascending=False)

In [39]:
avg_abund_each_domain = domain_summary.merge(std_count, on="Domain")

### mean and standard deviation of domain average abundance for each protein domain:

In [42]:
avg_abund_each_domain

,Domain,domain_average_abundance,count_domain,domain_abundance_std
0,ubiquitin,9970.828250,14,532.688848
1,Tropomyosin,4653.332000,3,663.757852
2,Tubulin,4617.474333,7,424.086615
3,Tubulin_C,4617.474333,7,424.086615
4,zf-C2H2,2261.611833,620,18.429448
...,...,...,...,...
432,Trimer_CC,1.393000,1,0.108894
433,FAM76,1.012000,1,0.050912
434,GM130_C,0.835000,1,0.104652
435,FAM72,0.306000,3,0.041569


In [43]:
# write to .csv file:
# avg_abund_each_domain.to_csv("avg_abundance_protein_domains.csv", index=False)

### compute the percentile rank values as above (One table)

In [44]:
avg_abund_each_domain["percentile_rank"] = avg_abund_each_domain["domain_average_abundance"].rank(pct=True, ascending=True) * 100

In [45]:
avg_abund_each_domain_ranks = avg_abund_each_domain.sort_values("percentile_rank", ascending=False)

In [52]:
avg_abund_each_domain_ranks = avg_abund_each_domain_ranks[["Domain", "domain_average_abundance", "domain_abundance_std",
    "count_domain", "percentile_rank"
                                                          ]]
avg_abund_each_domain_ranks.head(10)

,Domain,domain_average_abundance,domain_abundance_std,count_domain,percentile_rank
0,ubiquitin,9970.828250,532.688848,14,100.000000
1,Tropomyosin,4653.332000,663.757852,3,99.771167
2,Tubulin,4617.474333,424.086615,7,99.427918
3,Tubulin_C,4617.474333,424.086615,7,99.427918
4,zf-C2H2,2261.611833,18.429448,620,99.084668
5,Spectrin,2088.733500,56.975129,31,98.855835
6,RRM,1948.366333,150.199195,15,98.627002
7,EF-hand,1634.424000,725.399604,6,98.398169
8,UDP-g_GGTase,1621.509000,326.230077,2,98.169336
9,Sushi,1439.700500,84.217423,5,97.940503


In [53]:
# write to .csv file:
avg_abund_each_domain_ranks.to_csv("B2task_avg_abund_protein_domains_ranks.csv", index=False)